<a href="https://colab.research.google.com/github/griffithpoindexter/PageRank-HITS-SimRank/blob/master/CIS531_731_Fall_HW2_Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Homework 2 

By: Griffith Poindexter 


## General info
The objective of the code below was to fulfill the assignment requirements listed in the Homework 2 word file
	
## Technologies
Project is created with:
* Spark 
	
##Files needed 
* https://www.gutenberg.org/files/66057/66057-0.txt
* https://www.gutenberg.org/files/66057/66057-0.txt


In [ ]:
####### CHANGES MADE BY INSTRUCTOR" ########
!pip install -q pyspark
!wget -q https://www.gutenberg.org/files/66057/66057-0.txt

In [ ]:
#import appropriate libraries and create Spark and SQL session 
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [ ]:
# create removePunctuation function to clean data 
import re
import string
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub('', text).lower().strip()
    
print (removePunctuation('YŪGAO'))
print (removePunctuation(' No under_score!'))

yūgao
no underscore


In [ ]:
# create removePunctuation function that does not lower case items, removes numbers, and removes roman numerals 
import re
import string
def removePunctuationupper(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    replace_dashes = text.replace("-"," ")
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    data = regex.sub('', replace_dashes).strip()
    remove_numbers = re.compile('[0-9]')
    data_no_numbers = remove_numbers.sub(' ',data)
    remove_numerals = re.compile(r'(?:^|\s)(IX|IV|V?I{0,3})(?:\s|$)')
    return remove_numerals.sub(' ',data_no_numbers)

#Notice the difference between the 
print (removePunctuationupper('YŪGAO12'))
print (removePunctuationupper('Lady-in-Perpetual-Attendance'))

YŪGAO 
Lady in Perpetual Attendance


In [ ]:
#create function to remove additional types of punctuation that may have been missed by the removePunctuation function 
import sys
import unicodedata
import re

punc_list=[]

for i in range(0, sys.maxunicode + 1):
  if unicodedata.category(chr(i)) in ('Pc', 'Pd', 'Pe', 'Pf', 'Pi', 'Po', 'Ps'):
    punc_list.append(chr(i))

punc_string=''.join(punc_list)

def removeUniPunctuation(text):
    regex = re.compile('[%s]' % re.escape(punc_string))
    return regex.sub(' ', text).strip()

In [ ]:
#import files and parse to 10 different mappers 
from pyspark.sql.functions import regexp_extract, col

fileName = sc.textFile('/content/sample_data/66057-0.txt',10)

remove_numerals = re.compile(r'(?:^|\s)(IX|IV|V?I{0,3})(?:\s|$)')

#first attempt to remove punctuation from data 
fileName2 = sc.textFile('/content/sample_data/66057-0.txt',10).map(lambda x: removeUniPunctuation(x))

#split all words into their own location using .split 
all_words_split = fileName.flatMap(lambda x: x.split())


#search for uppercase letters using regex, map roman number data cleaning logic to strings if the length of the value is > 1, split data into individual cells and count occurrences  
total_uppercase = fileName2.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count()

#search for lowercase letters using regex, map roman number data cleaning logic to strings if the length of the value is > 1, split data into individual cells and count occurrences  
total_lowercase = fileName2.flatMap(lambda x: re.findall(r'\b[a-z]+(?:\s+[a-z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count()

print('total number of uppercase words: ', total_uppercase)
print('total number of lowercase words: ', total_lowercase)

total number of uppercase words:  1403
total number of lowercase words:  89356


In [ ]:
#total count of all words in filename - 1. filename2 has logic that incorrectly expands the total word count, so it cannot be used. 
total_word_count  = all_words_split.count()
total_word_count

101277

In [ ]:
#read data accross 10 mappers 
#join string into one single line of text 
#split texts based on CHAPTER # and Appendix 3 

import re

book = sc.textFile('/content/sample_data/66057-0.txt',10).collect()

book_single_string = ' '.join(book)

chapters = re.split('CHAPTER \w{1,4} | APPENDIX \w{1}',book_single_string)


In [ ]:
#parralize chapter data into their own RDD for analysis 
intro = sc.parallelize([chapters[0]])
chapter1 = sc.parallelize([chapters[1]])
chapter2 = sc.parallelize([chapters[2]])
chapter3 = sc.parallelize([chapters[3]])
chapter4 = sc.parallelize([chapters[4]])
chapter5 = sc.parallelize([chapters[5]])
chapter6 = sc.parallelize([chapters[6]])
chapter7 = sc.parallelize([chapters[7]])
chapter8 = sc.parallelize([chapters[8]])
chapter9 = sc.parallelize([chapters[9]])
appendix = sc.parallelize([chapters[10] + chapters[11]])

#run punctuation logic and place word values into their own lists location 
intro_rdd_word_count = intro.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter1_rdd_word_count = chapter1.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter2_rdd_word_count = chapter2.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter3_rdd_word_count = chapter3.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter4_rdd_word_count = chapter4.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter5_rdd_word_count = chapter5.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter6_rdd_word_count = chapter6.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter7_rdd_word_count = chapter7.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter8_rdd_word_count = chapter8.map(removeUniPunctuation).flatMap(lambda x: x.split())
chapter9_rdd_word_count = chapter9.map(removeUniPunctuation).flatMap(lambda x: x.split())
appendix_rdd_word_count = appendix.map(removeUniPunctuation).flatMap(lambda x: x.split())


#percentage of uppercase values by introduction, chapter, appendix 
print('percentage of uppercase values in introduction: ', round(intro_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 1: ',round(chapter1_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 2: ',round(chapter2_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 3: ',round(chapter3_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 4: ',round(chapter4_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 5: ',round(chapter5_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 6: ',round(chapter6_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 7: ',round(chapter7_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 8: ',round(chapter8_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in chapter 9: ',round(chapter9_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))
print('percentage of uppercase values in appendix: ',round(appendix_rdd_word_count.flatMap(lambda x: re.findall(r'\b[A-Z]+(?:\s+[A-Z]+)*\b',x)).map(lambda x: remove_numerals.sub('',x) if len(x) > 1 else x).flatMap(lambda x: x.split()).count() / total_uppercase * 100,2))



percentage of uppercase values in introduction:  6.27
percentage of uppercase values in chapter 1:  4.35
percentage of uppercase values in chapter 2:  18.96
percentage of uppercase values in chapter 3:  2.71
percentage of uppercase values in chapter 4:  13.11
percentage of uppercase values in chapter 5:  16.18
percentage of uppercase values in chapter 6:  7.7
percentage of uppercase values in chapter 7:  4.21
percentage of uppercase values in chapter 8:  2.14
percentage of uppercase values in chapter 9:  8.55
percentage of uppercase values in appendix:  16.11


In [ ]:
#import schema related functions 
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.functions import lower, col

#look for these words in the dataframe 
word_search = ['Aoi','Emperor','Genji','Kiritsubo', 'Kōkiden','Koremitsu']

#create datafrme of wordsearch values 
df_word_search = sqlContext.createDataFrame(word_search, StringType())

#create data frame with remove punctuation - upper function 
dfintro = sqlContext.createDataFrame(intro_rdd_word_count.map(removePunctuationupper), StringType())
df1 = sqlContext.createDataFrame(chapter1_rdd_word_count.map(removePunctuationupper), StringType())
df2 = sqlContext.createDataFrame(chapter2_rdd_word_count.map(removePunctuationupper), StringType())
df3 = sqlContext.createDataFrame(chapter3_rdd_word_count.map(removePunctuationupper), StringType())
df4 = sqlContext.createDataFrame(chapter4_rdd_word_count.map(removePunctuationupper), StringType())
df5 = sqlContext.createDataFrame(chapter5_rdd_word_count.map(removePunctuationupper), StringType())
df6 = sqlContext.createDataFrame(chapter6_rdd_word_count.map(removePunctuationupper), StringType())
df7 = sqlContext.createDataFrame(chapter7_rdd_word_count.map(removePunctuationupper), StringType())
df8 = sqlContext.createDataFrame(chapter8_rdd_word_count.map(removePunctuationupper), StringType())
df9 = sqlContext.createDataFrame(chapter9_rdd_word_count.map(removePunctuationupper), StringType())
dfappendix = sqlContext.createDataFrame(appendix_rdd_word_count.map(removePunctuationupper), StringType())

#count the occurends of the word search values by chapter 
print('introduction count: ',dfintro[dfintro['value'].isin(word_search)].count())
print('chapter 1 count: ', df1[df1['value'].isin(word_search)].count())
print('chapter 2 count: ',df2[df2['value'].isin(word_search)].count())
print('chapter 3 count: ',df3[df3['value'].isin(word_search)].count())
print('chapter 4 count: ',df4[df4['value'].isin(word_search)].count())
print('chapter 5 count: ',df5[df5['value'].isin(word_search)].count())
print('chapter 6 count: ',df6[df6['value'].isin(word_search)].count())
print('chapter 7 count: ',df7[df7['value'].isin(word_search)].count())
print('chapter 8 count: ',df8[df8['value'].isin(word_search)].count())
print('chapter 9 count: ',df9[df9['value'].isin(word_search)].count())
print('appendix count: ',dfappendix[dfappendix['value'].isin(word_search)].count())


introduction count:  46
chapter 1 count:  83
chapter 2 count:  93
chapter 3 count:  30
chapter 4 count:  155
chapter 5 count:  156
chapter 6 count:  85
chapter 7 count:  131
chapter 8 count:  50
chapter 9 count:  189
appendix count:  14


In [ ]:
#read in file name and run removePunctuation - upper to clean data 
filename_with_data_cleaned_when_create = fileName.map(removePunctuationupper)

#split word values into seperate list locations 
file_name_prep_for_unique_count = filename_with_data_cleaned_when_create.flatMap(lambda x: x.split())

#count words, reduce by key values and count them to obtain unique word values 
unique_count_of_words = file_name_prep_for_unique_count.map(lambda word: (word,1)).reduceByKey(lambda a,b: a+b).count()

print('number of unique words: ', unique_count_of_words)



number of unique words:  9379


Homework-2
Question-3: Fill out the right hand side and/or below #TODO comments
Setup the system and environment variables

You need to work on Project Gutenberg edition of Part I (the first 9 chapters) of Arthur Waley’s translation of Murasaki Shikibu’s The Tale of Genji (https://www.gutenberg.org/files/66057/66057-0.txt)

Sort the unique words in the book based on both words and counts in assending and descending order.

Split the 61 chapters (dataframe) into 6 bins of 10 chapters in each: Ch-1 to Ch-10, Ch-11 to Ch-20, Ch-21 to Ch-30, Ch-31 to Ch-40, Ch-41 to Ch-50, Ch-51 to End. The code to split the book in to a dataframe with chapter-number and text column is given.

Randomly shuffle the dataframe containing 61 chapters.

In [ ]:
####### CHANGES MADE BY INSTRUCTOR" ########
!pip install -q pyspark
!wget -q https://www.gutenberg.org/files/1342/1342-0.txt

####### CHANGES MADE BY INSTRUCTOR" ########
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

####### CHANGES MADE BY INSTRUCTOR" ########
import os.path
baseDir = os.path.join('data')
inputPath = os.path.join('/content/1342-0.txt')
fileName = os.path.join(baseDir, inputPath)
print(fileName)

/content/1342-0.txt


Drop files to upload them to session storage
Disk
70.19 GB available
Homework-2
Question-3: Fill out the right hand side and/or below #TODO comments
Setup the system and environment variables

You need to work on Project Gutenberg edition of Part I (the first 9 chapters) of Arthur Waley’s translation of Murasaki Shikibu’s The Tale of Genji (https://www.gutenberg.org/files/66057/66057-0.txt)

Sort the unique words in the book based on both words and counts in assending and descending order.

Split the 61 chapters (dataframe) into 6 bins of 10 chapters in each: Ch-1 to Ch-10, Ch-11 to Ch-20, Ch-21 to Ch-30, Ch-31 to Ch-40, Ch-41 to Ch-50, Ch-51 to End. The code to split the book in to a dataframe with chapter-number and text column is given.

Randomly shuffle the dataframe containing 61 chapters.

[7]
####### CHANGES MADE BY INSTRUCTOR" ########
!pip install -q pyspark
!wget -q https://www.gutenberg.org/files/1342/1342-0.txt

####### CHANGES MADE BY INSTRUCTOR" ########
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

####### CHANGES MADE BY INSTRUCTOR" ########
import os.path
baseDir = os.path.join('data')
inputPath = os.path.join('/content/1342-0.txt')
fileName = os.path.join(baseDir, inputPath)
print(fileName)
[8]
[9]
/content/1342-0.txt
Sorting: orderBy() function is described at https://spark.apache.org/docs/latest/api/python/pyspark.sql.html
Load the text file using SparkContext (Provided below)
get the word counts RDD
Convert word counts RDD to dataframe and print 5 entries
Sort dataframe by words and counts in assending order and print 5 entries for both
Sort dataframe by words and counts in descending order and print 5 entries for both


In [ ]:
####### CHANGES MADE BY INSTRUCTOR" ########
# Remove Unicode Punctuation

import sys
import unicodedata
import re

punc_list=[]

for i in range(0, sys.maxunicode + 1):
  if unicodedata.category(chr(i)) in ('Pc', 'Pd', 'Pe', 'Pf', 'Pi', 'Po', 'Ps'):
    punc_list.append(chr(i))

punc_string=''.join(punc_list)

def removeUniPunctuation(text):
    regex = re.compile('[%s]' % re.escape(punc_string))
    return regex.sub('', text).strip()
# Test
print (removeUniPunctuation('s wise: “Merlin”, so far as my'))

s wise Merlin so far as my


In [ ]:
from pyspark.sql.functions import col ####### CHANGES MADE BY INSTRUCTOR" ########
from pyspark.sql import Row ####### CHANGES MADE BY INSTRUCTOR" ########
from pyspark.sql.types import * ####### CHANGES MADE BY INSTRUCTOR" ########
from pyspark.sql.functions import desc 

# TODO: specify the path to the input file (String)
ipFilePath = '/content/sample_data/1342-0.txt'
words = sc.textFile(ipFilePath).flatMap(lambda line: line.split(" "))
print (type(words))
#print words.take(20)

# get word-count RDD
# TODO
wordCounts = words.map(removeUniPunctuation).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
#print wordCounts.take(20)

# Convert word-count RDD to dataframe with 2 columns ["word","count"]
# TODO
wcDF = sqlContext.createDataFrame(wordCounts, ["word","count"])
print (type(wcDF))
wcDF.show(5)

# sorting in assending order
print ('sorted by word: assending')
# TODO
wcDF.orderBy(["word", "count"], ascending=[0, 1]).show()

print ('sorted by count: assending')
# TODO
wcDF.orderBy(["count", "word"], ascending=[0, 1]).show()

# sorting in descending order
print ('sorted by word: descending')
# TODO
wcDF.orderBy(desc("word"), "count").show(10)

print ('sorted by count: descending')
# TODO
wcDF.orderBy(desc("count"), "word").show(10)



<class 'pyspark.rdd.PipelinedRDD'>
<class 'pyspark.sql.dataframe.DataFrame'>
+-------+-----+
|   word|count|
+-------+-----+
|    The|  285|
|Project|   84|
|     of| 3698|
|  Pride|    5|
|   Jane|  262|
+-------+-----+
only showing top 5 rows

sorted by word: assending
+-----------------+-----+
|             word|count|
+-----------------+-----+
|            éclat|    1|
|           youths|    1|
|            youth|    9|
|       yourselves|    2|
|      yourselfand|    1|
|         yourself|   49|
|            yours|   14|
|             your|  419|
|         youngest|   13|
|          younger|   29|
|            young|  127|
|            youll|    1|
|           youhow|    1|
|           youhad|    1|
|           youbut|    2|
|            yoube|    1|
|           youand|    2|
|            youMr|    1|
|              you| 1196|
|yieldingcertainly|    1|
+-----------------+-----+
only showing top 20 rows

sorted by count: assending
+----+-----+
|word|count|
+----+-----+
|    |72902|

In [ ]:
import unicodedata
import re
from pyspark.ml.feature import Bucketizer

################## Code for getting chapters out of book #####################

def normalize_text(text):
    #if not isinstance(text, str): ####### CHANGES MADE BY INSTRUCTOR" ########
    #text = unicode(text, 'utf-8')
    #text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore') 
    return text

def remove_white_spaces(text):
    text = re.sub('\s+', ' ', text).strip()
    return text

with open('/content/sample_data/1342-0.txt', 'r') as content_file:
    content_read = content_file.read()


content = remove_white_spaces(normalize_text(content_read))

start_str = 'Chapter 60 Chapter 61' ####### CHANGES MADE BY INSTRUCTOR" ########
end_str = 'End of the Project Gutenberg EBook of Pride and Prejudice, by Jane Austen'
actual_content = content[content.find(start_str)+len(start_str):content.rfind(end_str)]



chapters = re.split(" Chapter [0-9]+ ", actual_content, flags=re.IGNORECASE)[1:]

ch_data = []
for i in range(1,1+len(chapters)):
    ch_data.append((i,chapters[i-1]))

##############################################################################

# Get the dataframe out of list of tuples with columns as ["chapter", "text"]
# TODO

df = sqlContext.createDataFrame(ch_data, ["chapter", "text"])

#chapter_list = df.withColumn("chapter",col("chapter").cast('float')).collect()

df2 = df.withColumn("chapter",col("chapter").cast('float'))

#chapter_list = [chapter_list[x][0] for x in range(len(chapter_list))]

# do the binning using Bucketizer
# TODO

bucketizer = Bucketizer(splits= [0,21,41,float('Inf')], inputCol= "chapter", outputCol = 'buckets')

df_buck = bucketizer.setHandleInvalid("keep").transform(df2) 

df_buck.show(61)

+-------+--------------------+-------+
|chapter|                text|buckets|
+-------+--------------------+-------+
|    1.0|It is a truth uni...|    0.0|
|    2.0|Mr. Bennet was am...|    0.0|
|    3.0|Not all that Mrs....|    0.0|
|    4.0|When Jane and Eli...|    0.0|
|    5.0|Within a short wa...|    0.0|
|    6.0|The ladies of Lon...|    0.0|
|    7.0|Mr. Bennet’s prop...|    0.0|
|    8.0|At five o’clock t...|    0.0|
|    9.0|Elizabeth passed ...|    0.0|
|   10.0|The day passed mu...|    0.0|
|   11.0|When the ladies r...|    0.0|
|   12.0|In consequence of...|    0.0|
|   13.0|“I hope, my dear,...|    0.0|
|   14.0|During dinner, Mr...|    0.0|
|   15.0|Mr. Collins was n...|    0.0|
|   16.0|As no objection w...|    0.0|
|   17.0|Elizabeth related...|    0.0|
|   18.0|Till Elizabeth en...|    0.0|
|   19.0|The next day open...|    0.0|
|   20.0|Mr. Collins was n...|    0.0|
|   21.0|The discussion of...|    1.0|
|   22.0|The Bennets were ...|    1.0|
|   23.0|Elizabeth was si

Shuffling: https://stackoverflow.com/questions/43637625/how-to-shuffle-the-rows-in-a-spark-dataframe
Print 5 entries of the dataframe containing all 61 chapters
Use pyspark.sql.functions.rand function for randomly shuffling the dataframe containing all 61 chapters and print 5 entries

In [ ]:
from pyspark.sql.functions import rand 
# print 5 entries of the dataframe containing all 61 chapters
# TODO
df_buck.show(5)


# randomly shuffle the dataframe and print 5 entries
# TODO

df_buck.orderBy(rand()).show(5)

+-------+--------------------+-------+
|chapter|                text|buckets|
+-------+--------------------+-------+
|    1.0|It is a truth uni...|    0.0|
|    2.0|Mr. Bennet was am...|    0.0|
|    3.0|Not all that Mrs....|    0.0|
|    4.0|When Jane and Eli...|    0.0|
|    5.0|Within a short wa...|    0.0|
+-------+--------------------+-------+
only showing top 5 rows

+-------+--------------------+-------+
|chapter|                text|buckets|
+-------+--------------------+-------+
|   40.0|Elizabeth’s impat...|    1.0|
|   30.0|Sir William staye...|    1.0|
|   26.0|Mrs. Gardiner’s c...|    1.0|
|   61.0|Happy for all her...|    2.0|
|   19.0|The next day open...|    0.0|
+-------+--------------------+-------+
only showing top 5 rows

